In [2]:
import pandas as pd
import random
import csv
import numpy as np
from pandas.api.types import is_string_dtype
import random
from datetime import datetime, timedelta
from collections import defaultdict

In [ ]:
equipment_df = pd.read_csv("equipment.csv")
stage_df = pd.read_csv("stage.csv")
festival_df = pd.read_csv("festival.csv")
subgenre_df = pd.read_csv("subgenre.csv")
performer_df = pd.read_csv("performer.csv")
staff_df = pd.read_csv("staff.csv")
event_df = pd.read_csv("generated/event.csv")
performance_type_df = pd.read_csv("performance_type.csv")
artist_has_performer_df = pd.read_csv("artist_has_performer.csv")
artist_df = pd.read_csv("artist.csv")
visitor_df = pd.read_csv("visitor.csv")
ticket_type_df = pd.read_csv("ticket_type.csv")
seller_df = pd.read_csv("seller.csv")
payment_method_df = pd.read_csv("payment_method.csv")
ticket_df = pd.read_csv("ticket.csv")
visitor_df = pd.read_csv("visitor.csv")
performance_df = pd.read_csv("performance.csv")

In [ ]:
#equipment_has_stage
equipment_has_stage = {
    "Equipment_Name":[],
    "Stage_Stage_id":[],
    "Quantity":[]
}
equip_name = equipment_df["Name"].tolist()
stage_id = stage_df["Stage_id"].tolist()

for stage in stage_id:
  for equip in equip_name:
    q = random.randint(2,15)
    equipment_has_stage["Stage_Stage_id"].append(stage)
    equipment_has_stage["Equipment_Name"].append(equip)
    equipment_has_stage["Quantity"].append(q)

pd.DataFrame(equipment_has_stage).to_csv("generated/equipment_has_stage.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')

                         Table 1
0  Name;Description;Image_id;;;;
1                μικροφωνο;;;;;;
2               φωτιστικό ;;;;;;
3                  καλωδιο;;;;;;


'\'\'\nequip_name = equipment_df["Name"].tolist()\nstage_id = stage_df["Stage_id"].tolist()\n\n  \nfor stage in stage_id:\n  for equip in equip_name:\n    q = random.randint(2,15)\n    equipment_has_stage["Stage_Stage_id"].append(stage)\n    equipment_has_stage["Equipment_Name"].append(equip)\n    equipment_has_stage["Quantity"].append(q)\n\npd.DataFrame(equipment_has_stage).to_csv("generated/equipment_has_stage.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar=\'"\')\n'

In [ ]:
#performer_has_subgenre
performer_has_subgenre = {
    "Performer_Performer_id":[],
    "Subgenre_Subgenre_id":[]
}

perf_id = performer_df["Performer_id"].tolist()
sub_id = subgenre_df["Subgenre_id"].tolist()

for performer in perf_id:
  for sub in range(1, random.randint(2, 6)):
    performer_has_subgenre["Performer_Performer_id"].append(performer)
    performer_has_subgenre["Subgenre_Subgenre_id"].append(sub)

pd.DataFrame(performer_has_subgenre).to_csv("generated/performer_has_subgenre.csv", index = False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')

In [ ]:
#events
event_records = []
used_slots = {}  # key = (stage_id, date)

for _, fest in festival_df.iterrows():
    fest_days = pd.date_range(start=fest["StartDate"], end=fest["EndDate"])
    for day in fest_days:
        stage_id = random.choice(stage_df["Stage_id"].tolist())
        if (stage_id, day.date()) in used_slots:
            continue  # skip overlapping
        used_slots[(stage_id, day.date())] = True
        event_records.append({
            "Event_id": len(event_records)+1,
            "Date": day.date(),
            "Festival_Festival_id": fest["Festival_id"],
            "Stage_Stage_id": stage_id
        })

event_df = pd.DataFrame(event_records)
event_df.to_csv("generated/event.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')

In [ ]:
#event_has_staff
staff_roles = staff_df.groupby("Role_name")["Staff_id"].apply(list).to_dict()
stage_capacity = stage_df.set_index("Stage_id")["Capacity"].to_dict()

event_has_staff = {
    "Event_Event_id": [],
    "Staff_Staff_id": []
}

# Χάρτης διαθεσιμότητας προσωπικού: {Staff_id: [ημερομηνίες]}
staff_availability = defaultdict(set)

for _, event in event_df.iterrows():
    event_id = event["Event_id"]
    event_date = pd.to_datetime(event["Date"]).date()
    stage_id = event["Stage_Stage_id"]
    cap = stage_capacity[stage_id]

    required = {
        "Ασφάλεια": max(1, int(cap * 0.05)),
        "Βοηθητικό": max(1, int(cap * 0.02)),
        "Τεχνικό": random.randint(2, 4)
    }

    for role, count in required.items():
        available_staff = [
            sid for sid in staff_roles.get(role, [])
            if event_date not in staff_availability[sid]
        ]
        selected = random.sample(available_staff, min(count, len(available_staff)))

        for sid in selected:
            event_has_staff["Event_Event_id"].append(event_id)
            event_has_staff["Staff_Staff_id"].append(sid)
            staff_availability[sid].add(event_date)  # Mark as busy that day

pd.DataFrame(event_has_staff).to_csv("generated/event_has_staff.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')


In [ ]:
#performance
# βοηθοί
performance = {
    "Performance_id": [],
    "StartTime": [],
    "DurationINT": [],
    "BreakDurationINT": [],
    "Event_Event_id": [],
    "Performer_Performer_id": [],
    "Performance_type_Type": []
}

performance_id = 1

# 1. κρατάμε για κάθε performer πότε εμφανίστηκε (για overlap)
performer_schedule = defaultdict(list)  # {performer_id: [ (start, end) ]}
# 2. για κάθε performer: έτη συμμετοχής
performer_years = defaultdict(set)  # {performer_id: set(έτη)}
# 3. για κάθε artist: ποια groups/performers ανήκει
artist_performer_map = defaultdict(list)
for _, row in artist_has_performer_df.iterrows():
    artist_performer_map[row["Artist_Artist_id"]].append(row["Performer_Performer_id"])

# 4. helper για να ελέγξουμε overlap
def is_available(p_id, new_start, new_end):
    for s, e in performer_schedule[p_id]:
        if not (new_end <= s or new_start >= e):
            return False
    return True

# για κάθε event
for _, event in event_df.iterrows():
    event_id = event["Event_id"]
    event_date = pd.to_datetime(event["Date"]).date()
    festival_year = festival_df[festival_df["Festival_id"] == event["Festival_Festival_id"]]["YearINT"].values[0]

    # ξεκινάμε χρονικά από μια βάση ώρας
    current_time = datetime.combine(event_date, datetime.strptime("10:00", "%H:%M").time())

    max_performers = random.randint(2, 4)
    chosen_performers = []

    attempts = 0
    while len(chosen_performers) < max_performers and attempts < 100:
        attempts += 1
        candidate = random.choice(performer_df["Performer_id"].tolist())

        # 1. ελεγξε "μέχρι 3 συνεχή έτη"
        years = performer_years[candidate]
        if any(
            all((festival_year - i) in years for i in range(1, 4))
        ):
            continue  # έχει ήδη 3 συνεχόμενες συμμετοχές

        # 2. ελεγξε "διαθεσιμότητα" performer (και artists του αν είναι group)
        duration = random.randint(30, 180)
        break_time = random.randint(5, 30)
        start_time = current_time
        end_time = current_time + timedelta(minutes=duration)

        if not is_available(candidate, start_time, end_time):
            continue

        # αν είναι group, ελεγξε και artists
        artists_in_group = artist_has_performer_df[
            artist_has_performer_df["Performer_Performer_id"] == candidate
        ]["Artist_Artist_id"].tolist()

        valid = True
        for artist_id in artists_in_group:
            for performer_of_artist in artist_performer_map[artist_id]:
                if not is_available(performer_of_artist, start_time, end_time):
                    valid = False
                    break
            if not valid:
                break
        if not valid:
            continue

        # passed all checks
        chosen_performers.append(candidate)
        performance["Performance_id"].append(performance_id)
        performance["StartTime"].append(start_time)
        performance["DurationINT"].append(duration)
        performance["BreakDurationINT"].append(break_time)
        performance["Event_Event_id"].append(event_id)
        performance["Performer_Performer_id"].append(candidate)
        performance["Performance_type_Type"].append(random.choice(performance_type_df["Type"].tolist()))

        # ενημέρωσε χάρτες
        performer_schedule[candidate].append((start_time, end_time))
        performer_years[candidate].add(festival_year)
        for artist_id in artists_in_group:
            for pid in artist_performer_map[artist_id]:
                performer_schedule[pid].append((start_time, end_time))

        performance_id += 1
        current_time = end_time + timedelta(minutes=break_time)

pd.DataFrame(performance).to_csv("generated/performance.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')


In [ ]:
#performer
#artist_has_performer
# Αν δεν υπάρχει ο φάκελος
os.makedirs("generated", exist_ok=True)

# Διαβάζουμε τον πίνακα artist
artist_df = pd.read_csv("artist.csv")
artist_df = artist_df.sample(frac=1, random_state=42).reset_index(drop=True)

num_artists = len(artist_df)
num_group_artists = num_artists // 2  # 50% σε group

# Δημιουργούμε groups των 2-7 μελών μέχρι να συμπληρώσουμε το 50%
group_sizes = []
count = 0
while count < num_group_artists:
    size = random.randint(2, 7)
    if count + size > num_group_artists:
        break
    group_sizes.append(size)
    count += size

# IDs για group artists
group_artist_ids = artist_df.iloc[:sum(group_sizes)]
solo_artist_ids = artist_df.iloc[sum(group_sizes):]

# Τυχαίες λέξεις για group ονόματα
word_list = [
    "Echo", "Nova", "Electric", "Fusion", "Sonic", "Crimson", "Velvet", "Nebula",
    "Quantum", "Rogue", "Lunar", "Vibe", "Zenith", "Orbit", "Bliss", "Jungle", "Tiger","Boom", "Hell"
]

performers = []
artist_has_performer = []
performer_id = 1

# Solo artists
for _, row in solo_artist_ids.iterrows():
    performers.append({
        "Performer_id": performer_id,
        "Name": row["StageName"],
        "FormationDate": pd.to_datetime("2015-01-01") + pd.to_timedelta(random.randint(0, 3000), unit="D"),
        "Website": f"http://{row['StageName'].replace(' ', '').lower()}.com",
        "InstagramProfile": f"https://instagram.com/{row['StageName'].replace(' ', '').lower()}"
    })
    artist_has_performer.append({
        "Artist_Artist_id": row["Artist_id"],
        "Performer_Performer_id": performer_id
    })
    performer_id += 1

# Group artists
idx = 0
for size in group_sizes:
    members = group_artist_ids.iloc[idx:idx + size]
    group_name = " ".join(random.sample(word_list, k=random.choice([1, 2])))
    formation = pd.to_datetime("2010-01-01") + pd.to_timedelta(random.randint(0, 4000), unit="D")

    performers.append({
        "Performer_id": performer_id,
        "Name": group_name,
        "FormationDate": formation,
        "Website": f"http://{group_name.replace(' ', '').lower()}.com",
        "InstagramProfile": f"https://instagram.com/{group_name.replace(' ', '').lower()}"
    })

    for _, artist_row in members.iterrows():
        artist_has_performer.append({
            "Artist_Artist_id": artist_row["Artist_id"],
            "Performer_Performer_id": performer_id
        })

    performer_id += 1
    idx += size

# Save to CSVs
pd.DataFrame(performers).to_csv("generated/performer.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')
pd.DataFrame(artist_has_performer).to_csv("generated/artist_has_performer.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')


In [ ]:
#ticket

os.makedirs("generated", exist_ok=True)

ticket_data = []
buyer_has_ticket_data = []
ticket_id = 1

# Create a mapping for stage capacity
stage_capacity = dict(zip(stage_df["Stage_id"], stage_df["MaxCapacity"]))

# Helper to prevent duplicate ticket per event per visitor
visitor_event_tickets = set()

for _, event in event_df.iterrows():
    event_id = event["Event_id"]
    stage_id = event["Stage_Stage_id"]
    event_date = pd.to_datetime(event["Date"])
    max_capacity = stage_capacity[stage_id]

    # Define number of tickets to generate
    num_tickets = max_capacity

    # VIP = 10%
    num_vip = int(num_tickets * 0.1)
    num_regular = num_tickets - num_vip

    # Set base price randomly per event
    base_price = random.randint(40, 120)

    # Shuffle visitors
    visitors = visitor_df.sample(frac=1).reset_index(drop=True)

    used_visitors = 0

    for i in range(num_tickets):
        if used_visitors >= len(visitors):
            break  # Not enough visitors

        visitor_id = visitors.iloc[used_visitors]["Visitor_id"]
        used_visitors += 1

        # Ensure 1 ticket per visitor per event
        if (visitor_id, event_id) in visitor_event_tickets:
            continue
        visitor_event_tickets.add((visitor_id, event_id))

        # Determine type
        if i < num_vip:
            ticket_type = "VIP"
            price = base_price + 100
        else:
            ticket_type = "Regular"
            price = base_price

        purchase_date = event_date - timedelta(days=random.randint(1, 60))
        #seller_id = random.choice(seller_df["Seller_id"].tolist())
        payment_method = random.choice(payment_method_df["Method"].tolist())

        ticket_data.append({
            "Ticket_id": ticket_id,
            "PurchaseDate": purchase_date,
            "Price": price,
            "EANCode": f"{random.randint(1000000000000,9999999999999)}",
            "Event_Event_id": event_id,
            "Ticket_type_Type": ticket_type,
            "Payment_method_Method": payment_method,
            "Seller_Seller_id": None
        })

        ticket_id += 1

# Save files
pd.DataFrame(ticket_data).to_csv("generated/ticket.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')
pd.DataFrame(buyer_has_ticket_data).to_csv("generated/buyer_has_ticket.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')


In [ ]:
#buyer_has_ticket
buyer_has_ticket = []

# 1. Μελλοντικά events
future_events = event_df[pd.to_datetime(event_df["Date"]) > datetime.now()]
future_event_ids = future_events["Event_id"].tolist()

# 2. Μόνο διαθέσιμα εισιτήρια: για future events και με Seller_id == None (NaN)
available_tickets = ticket_df[
    (ticket_df["Event_Event_id"].isin(future_event_ids)) &
    (ticket_df["Seller_Seller_id"].isna())
]

# 3. Visitors
visitor_ids = visitor_df["Visitor_id"].tolist()

# 4. Τι έχει αγοράσει ήδη κάθε visitor
visitor_tickets = ticket_df.groupby("Visitor_Visitor_id")["Event_Event_id"].apply(set).to_dict()

# 5. Ανά μελλοντικό event
for event_id in future_event_ids:
    event_tickets = available_tickets[available_tickets["Event_Event_id"] == event_id]
    if event_tickets.empty:
        continue

    # Visitors που δεν έχουν ήδη εισιτήριο για αυτό το event
    eligible_visitors = [
        v_id for v_id in visitor_ids
        if v_id not in visitor_tickets or event_id not in visitor_tickets[v_id]
    ]

    chosen_visitors = random.sample(eligible_visitors, min(len(eligible_visitors), random.randint(3, 10)))

    for visitor_id in chosen_visitors:
        # Επιλογή τύπου ενδιαφέροντος
        if random.random() < 0.5:
            # Ενδιαφέρον για 1 συγκεκριμένο εισιτήριο
            ticket_row = event_tickets.sample(1).iloc[0]
            buyer_has_ticket.append({
                "Buyer_Visitor_id": visitor_id,
                "Ticket_Ticket_id": ticket_row["Ticket_id"],
                "Event_Event_id": None,
                "datetime": datetime.now() - timedelta(days=random.randint(1, 30))
            })
        else:
            # Ενδιαφέρον για ΟΛΑ τα διαθέσιμα του event
            for _, ticket_row in event_tickets.iterrows():
                buyer_has_ticket.append({
                    "Buyer_Visitor_id": visitor_id,
                    "Ticket_Ticket_id": None,
                    "Event_Event_id": event_id,
                    "datetime": datetime.now() - timedelta(days=random.randint(1, 30))
                })

# Αποθήκευση
pd.DataFrame(buyer_has_ticket).to_csv("generated/buyer_has_ticket.csv", index=False)


In [ ]:
#resale_queue
# Join ticket με event για να έχουμε την ημερομηνία του event
ticket_with_event = ticket_df.merge(
    event_df[["Event_id", "Date"]],
    left_on="Event_Event_id",
    right_on="Event_id"
)

# Φιλτράρουμε μόνο εισιτήρια:
# - που έχουν Seller (δηλαδή μπήκαν για resale)
# - και το event τους είναι μελλοντικό
ticket_on_resale = ticket_with_event[
    ticket_with_event["Seller_Seller_id"].notna() &
    (pd.to_datetime(ticket_with_event["Date"]) > datetime.now())
]

# Δημιουργούμε ResaleQueue
resale_queue = {
    "Ticket_Ticket_id": [],
    "TimestampDATETIME": []
}

for _, row in ticket_on_resale.iterrows():
    resale_queue["Ticket_Ticket_id"].append(row["Ticket_id"])
    resale_queue["TimestampDATETIME"].append(
        datetime.now() - timedelta(days=random.randint(1, 10))  # Ημερομηνία δήλωσης μεταπώλησης
    )

# Αποθήκευση
pd.DataFrame(resale_queue).to_csv("generated/resalequeue.csv", index=False)


In [ ]:
#ratings
os.makedirs("generated", exist_ok=True)

# Χάρτης event_id -> ημερομηνία
event_dates = pd.to_datetime(event_df.set_index("Event_id")["Date"]).to_dict()

# Χάρτης event -> performances
event_to_performances = performance_df.groupby("Event_Event_id")["Performance_id"].apply(list).to_dict()

ratings = {
    "Rating_id": [],
    "ArtistPerformanceINT": [],
    "SoundAndLightingINT": [],
    "StagePresenceINT": [],
    "OrganizationINT": [],
    "OverallImpressionINT": [],
    "Performance_Performance_id": [],
    "Visitor_Visitor_id": []
}

rating_id = 1

# Για κάθε ticket → ψάχνουμε valid events που έχουν περάσει
for _, row in ticket_df.iterrows():
    visitor_id = row["Visitor_Visitor_id"]
    event_id = row["Event_Event_id"]

    # Ελέγχουμε αν υπάρχει η ημερομηνία event και αν έχει ήδη πραγματοποιηθεί
    event_date = event_dates.get(event_id)
    if not event_date or event_date > datetime.now():
        continue  # Το event δεν έχει γίνει ακόμα

    # Παίρνουμε τις performances του event
    possible_performances = event_to_performances.get(event_id, [])
    if not possible_performances:
        continue

    # Επιλέγουμε τυχαία 1–3 performances για αξιολόγηση
    num_to_rate = random.randint(1, min(3, len(possible_performances)))
    rated_performances = random.sample(possible_performances, num_to_rate)

    for perf_id in rated_performances:
        ratings["Rating_id"].append(rating_id)
        ratings["ArtistPerformanceINT"].append(random.randint(1, 5))
        ratings["SoundAndLightingINT"].append(random.randint(1, 5))
        ratings["StagePresenceINT"].append(random.randint(1, 5))
        ratings["OrganizationINT"].append(random.randint(1, 5))
        ratings["OverallImpressionINT"].append(random.randint(1, 5))
        ratings["Performance_Performance_id"].append(perf_id)
        ratings["Visitor_Visitor_id"].append(visitor_id)
        rating_id += 1

# Αποθήκευση
pd.DataFrame(ratings).to_csv("generated/rating.csv", index=False)
